In [3]:
import xlrd
import pandas as pd

In [4]:
vols=pd.read_csv('../Outputs/outvol_Hagan.csv')

In [5]:
from scipy.stats import norm
import math

def black(F_T,K,expiry,vol,isCall,r=0):
    #option_value=0
    D=math.exp(-r*expiry)
    if expiry*vol==0.0:
        if isCall:
            option_value=max(F_T-K,0.0)
        else:
            option_value=max(K-F_T,0.0)
    else:
        d1=dPlusBlack(F_T,K,expiry,vol,r)
        d2=dPlusBlack(F_T,K,expiry,vol,r)
        if isCall:
            option_value=F_T*D*norm.cdf(d1)-K*D*norm.cdf(d2)
        else:
            option_value=K*D*norm.cdf(-d2)-F_T*D*norm.cdf(-d1)
    return option_value

def dPlusBlack(F_T,K,expiry,vol,r):
    D=math.exp(-r*expiry)
    d_Plus=(math.log(F_T*D/K)+(r+0.5*vol*vol)*expiry)/vol/math.sqrt(expiry)
    return d_Plus

def dMinusBlack(F_T,K,expiry,vol,r):
    d_Minus=dPlusBlack(F_T,K,expiry,vol,r)-vol*math.sqrt(expiry)
    return d_Minus

def find_ivol(option_price,F_T,K,expiry,r=0):
    sigma=0.20 #initial guess of sigma
    while sigma<1:
        black_implied=black(F_T,K,expiry,sigma,1,r)
        if option_price-black_implied<0.0001: #set precision of 0.0001
            return sigma
        print sigma,option_price,black_implied
        sigma+=0.01
    return "failture to find the right ivol!"


def bs_matrix(F_T,K,expiry,vol,isCall,r=0): #F_T,K,expiry are vector, vol is a matrix
    for i in len(F_T):
        for j in len(K):
            option_price[i][j] = black(F_T[i],K[j],expiry[i],vol[i][j],isCall,r=0)
    return option_price

In [6]:
file_input=xlrd.open_workbook('../Inputs/'+'market_data.xlsx')  # load market data
Market_data=file_input.sheet_by_name('Swaptions data')

In [11]:
     ######## set characteristics ###############################
strike_spreads=[]
j = 0
while True:
    try:
        strike_spreads.append(int(Market_data.cell(1, 3 + j).value))
        j = j + 1
    except:
        break
num_strikes = len(strike_spreads)

expiries = []
i = 0
while True:
    try:
        expiries.append(Market_data.cell(2 + i, 1).value)
        i = i + 1
    except:
        break

tenors = []
i = 0
while True:
    try:
        tenors.append(Market_data.cell(2 + i, 0).value)
        i = i + 1
    except:
        break

    # to create the ATM forward rates
F = []
i = 0
while True:
    try:
        F.append(Market_data.cell(2 + i, 2).value)
        i = i + 1
    except:
        break

    # to create the strike grid
K = np.zeros((len(F), num_strikes))
for i in range(len(F)):
    for j in range(num_strikes):
        K[i][j] = F[i] + 0.0001 * (strike_spreads[j])

    # to create market volatilities
MKT = np.zeros((len(F), num_strikes))
for i in range(len(F)):
    for j in range(num_strikes):
        MKT[i][j] = Market_data.cell(2 + i, 3 + j).value

IndentationError: unindent does not match any outer indentation level (<ipython-input-11-7a270da9241e>, line 6)